In [1]:
!pip install scapy pandas matplotlib seaborn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 14.1 MB/s eta 0:00:00


In [2]:
from scapy.all import rdpcap, DNS, DNSQR, IP
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

PCAP_PATH = "traffic.pcap"  # если файл называется иначе — поменяй имя здесь

In [3]:
packets = rdpcap(PCAP_PATH)

rows = []

for pkt in packets:
    # интересуют только пакеты с DNS
    if DNS in pkt and IP in pkt:
        dns = pkt[DNS]
        ip = pkt[IP]

        # qr = 0 -> запрос, qr = 1 -> ответ
        is_response = dns.qr == 1

        query_name = None
        query_type = None

        if dns.qd is not None and isinstance(dns.qd, DNSQR):
            query_name = dns.qd.qname.decode(errors="ignore") if isinstance(dns.qd.qname, bytes) else dns.qd.qname
            query_type = dns.qd.qtype

        answers = []
        if is_response and dns.ancount > 0:
            # пробегаем по всем answer-записям
            for i in range(dns.ancount):
                rr = dns.an[i]
                # A-запись (IPv4)
                if rr.type == 1 and hasattr(rr, "rdata"):
                    answers.append(str(rr.rdata))

        rows.append(
            {
                "time": pd.to_datetime(pkt.time, unit="s"),
                "src_ip": ip.src,
                "dst_ip": ip.dst,
                "query_name": query_name,
                "query_type": query_type,
                "is_response": is_response,
                "answers": ",".join(answers) if answers else None,
            }
        )

df = pd.DataFrame(rows)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'traffic.pcap'